Installation
============
[Git](https://git-scm.com) is a prerequisite for installation. Also, development using either
[`venv`](https://docs.python.org/3/library/venv.html#creating-virtual-environments)
or [Docker](https://docs.docker.com/get-docker/)
is recommended.

`pytos2` CE is currently *not* yet available on [PyPI](https://pypi.org). To install, use
the following [`pip`](https://pypi.org/project/pip/) command:


In [ ]:
# PRODUCTION INSTALL
# %pip install python-dotenv git+https://gitlab.com/tufinps/pytos2-ce

# LOCAL DEVELOPMENT
%pip uninstall pytos2-ce -y
%pip install python-dotenv
%pip install ../../pytos2-ce

Setup
=====
`pytos2` CE recommends setting the following environment variables while using this library. They can either be set directly in the environment using a method of your choice, or you can use [python-dotenv](https://saurabh-kumar.com/python-dotenv/).

- `HOST`: The hostname 
- `USER`: The username 
- `PASS`: The password

These values can be editted in the `.env` file or overwritten in-line in the example below

In [ ]:
# %load init.py
from dotenv import dotenv_values
from pytos2.secureapp import Sa as SecureApp
from pytos2.securechange import Scw as SecureChange
from pytos2.securetrack import St as SecureTrack
from helpers import pp, pretty, ugly, pretty_json, ugly_json


# Loads our config
config = dotenv_values(".env")

# Override or Use Config Values (Strings must be empty to use config - any value in string will be truthy)
HOST = "" or config["HOST"]
USER = "" or config["USER"]
PASS = "" or config["PASS"]

# Instantiate Classes
secure_app = SecureApp(HOST, USER, PASS)
secure_change = SecureChange(HOST, USER, PASS)
secure_track = SecureTrack(HOST, USER, PASS)

### SecureChange
The following example will walk a SecureChange `Ticket` belonging to a
hypothetical `Workflow` from start to finish using `pytos2` CE,
showcasing its most-commonly-used features.

First, set up the `Scw` instance. This class instance will be
responsible for all SecureChange interactions:

#### Step 1
The first step of the sample `Workflow` we will be using is
enpictured here:

![Example Ticket Step #1](./assets/securechange-step-1.png)

Since our `Ticket` has ID #2335, let's fetch the `Ticket` by id
with the following:

In [ ]:
ticket = secure_change.get_ticket(2335)

With that being successful, let's move on to `Step` #2.

#### Step 2
![Example Ticket Step #2](./assets/securechange-step-2.png)

Now that we have the ticket, let's copy the Hyperlink field
"Website" from `Step` 1 to `Step` 2. (Technically, we'll be moving
the hyperlink from the first `Task` of `Step` 1 to the first
`Task` of `Step` 2, but since this example isn't using dynamic
assignment, we'll assume going forward that "`Step` N" is
referring to the first `Task` of "`Step` N"). First, let's
retrieve the value of the field on `Step 1` by name:

In [ ]:
hyperlink = ticket.steps[0].last_task.get_field("Website")

Great! Now, let's copy it to our current step's "Website" field:

In [ ]:
from pytos2.securechange.fields import Hyperlink

current_hyperlink: Hyperlink = ticket.last_task.get_field("Website")

current_hyperlink.url = hyperlink.url

ticket = ticket.save()

*N.B.:* We are making the result of the saved `Ticket` our new
working `Ticket`, as the prior `Ticket`-in-memory would be
invalidated at this point (since we just saved it; `Ticket`s do not currently update in-place).

If all has gone right, let's go back to SecureChange proper and
refresh our `Ticket`'s at `Step` 2:

![Example Ticket Step #2, Edit #1](./assets/securechange-step-2-edit-1.png)

Excellent. Now let's also programatically fill out the "Services" and "Service Expiration" fields, and advance the `Ticket`:

In [ ]:
from pytos2.securechange.fields import MultipleSelection, Date
from datetime import date

services: MultipleSelection = ticket.last_task.get_field("Services")
services.selected_options.append(services.options[0])
services.selected_options.append(services.options[2])


expiration: Date = ticket.last_task.get_field("Service Expiration")
expiration.date = date(2021, 12, 22)

ticket = ticket.advance()

Double-checking with SecureChange, we indeed see that `Step` 2 has
the corresponding fields updated, and that the `Ticket` has been
advanced to the final step, `Step` 3:

![Example Ticket Step #2, Edit #2](./assets/securechange-step-2-edit-2.png)

#### Step 3
`Step` 3 is the "Approval" step. In it, we'll accept this Service Change Request with an appropriate reason and check the "terms and conditions" checkbox programatically.

![Example Ticket Step #3](./assets/securechange-step-3.png)


In [ ]:
from pytos2.securechange.fields import ApproveReject

approval: ApproveReject = ticket.last_task.get_field("Approve")
approval.approved = True
approval.reason = "This service looks reasonable."

Now, let's check the checkbox.

 `pytos2` CE offers more than one
way to access `Step` fields. We've already seen
`ticket.task.get_field`, but there's also the `ticket.task.fields` property:

In [ ]:
from typing import List
from pytos2.securechange.fields import FieldType

fields: List[FieldType] = ticket.last_task.get_fields

Let's access and check the checkbox with this method:

In [ ]:
from pytos2.securechange.fields import Checkbox

checkbox: Checkbox = ticket.last_task.fields[-1]
checkbox.value = True

In production environments, you'll often want to use
`ticket.task.get_field` instead of get-by-index because if fields are ever reordered, your indexing code
will be broken. And while its entirely possible for field names to
change, they do so less frequently - and when they are, name changes are
easier to deal with than abstract indexes.

Finally, let's close `ticket`. All we need to do is advance the ticket, and if it's on the last step, SecureChange will close it automatically:

In [ ]:
ticket = ticket.advance()


As expected, visiting that same `Ticket` in SecureChange will show both the changes we've made, and that the `Ticket` is indeed closed!:

![Example Ticket, Closed](./assets/securechange-closed.png)